## Animation code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import matplotlib.patches as patches
from IPython.display import Image, display

# 1. DEFINE CLIQUE STRUCTURE 
meeting_groups_mat = [
    [0, 1, 2], [2, 3], [0, 4], [3, 4], [3, 6],
    [4, 5], [6, 7], [5, 7], [5, 8], [8, 9], [7, 9]
]

def get_automated_meeting_configs(Xall, cliques):
    """Detects the optimal (x,y) for each clique based on minimal dispersion."""
    configs = []
    for members in cliques:
        group_traj = Xall[members, :2, :]
        max_dist_at_t = []
        for t in range(Xall.shape[2]):
            pos_t = group_traj[:, :, t]
            diff = pos_t[:, None, :] - pos_t[None, :, :]
            dists = np.sqrt(np.sum(diff**2, axis=-1))
            max_dist_at_t.append(np.max(dists))
        
        t_meet = np.argmin(max_dist_at_t)
        meet_pos = np.mean(group_traj[:, :, t_meet], axis=0)
        label = f"R{'-R'.join([str(m+1) for m in members])} met"
        configs.append({'pos': meet_pos, 'label': label, 'members': members})
    return configs

def animate_with_dynamic_meetings(Xall, filename="RURAMCA_unicycle_final.gif"):
    num_agents, _, total_steps = Xall.shape
    tikz_colors = [(0.066, 0.443, 0.745), (0.866, 0.329, 0.000), (0.929, 0.694, 0.125),
                   (0.521, 0.086, 0.819), (0.231, 0.666, 0.196), (0.184, 0.745, 0.937), (0.819, 0.015, 0.545)]
    warmorange = (1, 0.6, 0)
    
    # Generate automated meeting data
    raw_configs = get_automated_meeting_configs(Xall, meeting_groups_mat)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.set_xlim(-1, 49); ax.set_ylim(-1, 21); ax.set_aspect('equal')
    ax.set_xlabel('z position'); ax.set_ylabel('y position')

    # Draw Obstacles and Target Regions
    obs_coords = [2.85, 20.85, 38.55] 
    for i, x_s in enumerate(obs_coords):
        ax.add_patch(patches.Rectangle((x_s, 8.6), 6.3, 2.8, facecolor=warmorange, alpha=0.9, edgecolor='black'))
        ax.text(x_s + 3.15, 10, f"☐ ¬O{i+1}", ha='center', va='center', fontsize=10, fontweight='bold')

    for i in range(10):
        c = tikz_colors[i % len(tikz_colors)]
        cx = 1.5 + (i * 5)
        # Diamond C_i
        ax.add_patch(patches.RegularPolygon((cx, 1.5), 4, radius=1.5, facecolor=c, alpha=0.2, edgecolor='black'))
        ax.text(cx, -0.3, f"R{i+1} Collects", ha='center', va='top', fontsize=8)
        # Square D_i
        ax.add_patch(patches.Rectangle((cx-1, 18), 2, 2, facecolor=c, alpha=0.35, edgecolor='black'))
        ax.text(cx, 20.3, f"R{i+1} Delivers", ha='center', va='bottom', fontsize=8)

    # Initialize Meeting Circles and Text with COLLISION AVOIDANCE
    meeting_patches = []
    meeting_texts = []
    
    # Sort configs by Y position to handle stacking
    sorted_configs = sorted(raw_configs, key=lambda x: x['pos'][1])
    
    placed_label_positions = [] # To track where we've put labels

    for cfg in sorted_configs:
        pos = cfg['pos']
        p = plt.Circle(pos, 0.7, color='green', alpha=0, zorder=4)
        
        # Logic to offset labels if points are too close
        y_offset = 1.1
        for prev_pos in placed_label_positions:
            if np.linalg.norm(pos - prev_pos) < 2.0: # Close proximity threshold
                y_offset += 1.0 # Stack higher
        
        t = ax.text(pos[0], pos[1] + y_offset, cfg['label'], 
                    fontsize=8, ha='center', alpha=0, fontweight='bold',
                    bbox=dict(facecolor='white', alpha=0, edgecolor='none', pad=0.5))
        
        placed_label_positions.append(pos)
        ax.add_patch(p)
        meeting_patches.append(p)
        meeting_texts.append(t)

    # Initialize Agents
    lines = [ax.plot([], [], lw=1.3, alpha=0.7, color=tikz_colors[i % 7])[0] for i in range(num_agents)]
    for i in range(num_agents):
        ax.plot(Xall[i, 0, 0], Xall[i, 1, 0], 's', color=tikz_colors[i % 7], markersize=4, alpha=0.4)

    # Agent Elements
    # Start positions (Squares)
    for i in range(num_agents):
        ax.plot(Xall[i, 0, 0], Xall[i, 1, 0], 's', color=tikz_colors[i % len(tikz_colors)], markersize=6, markeredgecolor='black')
    # lines = [ax.plot([], [], lw=1.5, color=tikz_colors[i%7])[0] for i in range(num_agents)]
    lines = [ax.plot([], [], lw=1.5, alpha=0.8, color=tikz_colors[i % len(tikz_colors)])[0] for i in range(num_agents)]
    heads = ax.quiver(Xall[:,0,0], Xall[:,1,0], np.cos(Xall[:,2,0]), np.sin(Xall[:,2,0]), 
                      color=[tikz_colors[i%7] for i in range(num_agents)], scale=35, pivot='mid')


    def update(frame):
        # Update Quivers
        # Update Agents
        X, Y = Xall[:, 0, frame], Xall[:, 1, frame]
        heads.set_offsets(np.stack([X, Y], axis=1))
        heads.set_UVC(np.cos(Xall[:, 2, frame]), np.sin(Xall[:, 2, frame]))
        for i in range(num_agents):
            lines[i].set_data(Xall[i, 0, :frame], Xall[i, 1, :frame])
        
        # Update Paths (frame+1 ensures head and tail are connected)
        for i in range(num_agents):
            lines[i].set_data(Xall[i, 0, :frame+1], Xall[i, 1, :frame+1])

        # Update Meeting Visibility
        threshold = 1.2
        for idx, cfg in enumerate(sorted_configs):
            agents_involved = cfg['members']
            pos = cfg['pos']
            dists = [np.linalg.norm(Xall[a, :2, frame] - pos) for a in agents_involved]
            
            if all(d < threshold for d in dists):
                meeting_patches[idx].set_alpha(0.3) 
                meeting_texts[idx].set_alpha(1.0)
                # Background white halo for readability
                meeting_texts[idx].get_bbox_patch().set_alpha(0.6)
            
        return lines + [heads] + meeting_patches + meeting_texts

    ani = FuncAnimation(fig, update, frames=range(0, total_steps, 1), blit=True, interval=50)
    ani.save(filename, writer=PillowWriter(fps=20))
    plt.close()
    display(Image(filename=filename))

# Execute
animate_with_dynamic_meetings(Xall) # change Xall for the corresponding scenario